In [1]:
#!pip install transformers==4.28.0 datasets evaluate sacrebleu torch - Requirements.txt should handle
#!pip install accelerate -U
from datasets import load_dataset, load_dataset_builder
from huggingface_hub import notebook_login

In [2]:
#Allows model upload (not strictly necessary, but may be useful in the future)
notebook_login()

In [3]:
#Load in EN-FR subset of OPUS books
books = load_dataset("opus_books", "en-fr")

Found cached dataset opus_books (C:/Users/ethan/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
#ds_builder = load_dataset_builder("rotten_tomatoes")
#ds_builder = load_dataset_builder("wmt14", 'fr-en')
#ds_builder.info.description
#ds_builder.info.features

In [5]:
#Entirety of OPUS books is a training dataset, so split into train:test with 80:20.
books = books["train"].train_test_split(test_size=0.2)

In [6]:
#Inspect data; the split is random.
books["train"][0]

{'id': '8394',
 'translation': {'en': 'I know that had I been a sanguine, brilliant, careless, exacting, handsome, romping child--though equally dependent and friendless--Mrs.',
  'fr': "Je sens que si j'avais été une enfant brillante, sans soin, exigeante, belle, folâtre, Mme Reed m'eût supportée plus volontiers, bien que je me fusse également trouvée sous sa dépendance et privée d'amis."}}

In [7]:
#Tokenise data - we must load the correct tokeniser for our model before we input parameters. This is based on SentencePiece.
from transformers import AutoTokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#NB: As long as data looks like {"input" : "XXXX" , "target" : "YYYY"}, it can be processed.

In [8]:
#Specify a preprocessing function which allows us to tokenise source and target languages correctly AND prime T5 with the correct prompt before each sentence
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]] 
    #We are essentially querying the dictionary here, whereby books["train"]["translation"][0]["en"] references the first English sentence above.
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True) #We will truncate sentences longer than 128 words long
    return model_inputs

#books["train"]["translation"][0]["en"]

In [9]:
#Apply this function over the training dataset over multiple elements simultaneously using the batched = True argument.
tokenized_books = books.map(preprocess_function, batched=True) 

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [10]:
#Create a batch of examples and dynamically pad to hit length of longest sentence per batch
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [11]:
#Load evaluation method
import evaluate
metric = evaluate.load("sacrebleu")

In [24]:
import numpy as np

def postprocess_text(preds, labels): 
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True) #Convert back into words
    print(decoded_preds[0])

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id) #Ignore padded labels added by the data collator to the test set
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    print(decoded_labels[0])

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels) #Remove leading and trailing spaces

    result = metric.compute(predictions=decoded_preds, references=decoded_labels) #BLEU score for provided input and references
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens) #Compute mean prediction length
    result = {k: round(v, 4) for k, v in result.items()} #Round score to 4dp
    return result

In [25]:
#Ready to download model
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint) #Model is 242MB in size

In [ ]:
training_args = Seq2SeqTrainingArguments( #Collects hyperparameters
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True, #Remove fp16 = True if not using CUDA
)

trainer = Seq2SeqTrainer( #Saves us from writing our own training loops
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()